# CSII 2024 # Exerercise 09: Seperation Principle, LQG, Stability Margins
&copy; 2024 ETH Zurich, Timm Grigat, Suno Dieckmann, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


## Description
TODO: Description of the notebook.

To start, run the following cells to install the necessary modules and import the libraries.

In [1]:
%pip install cs2solutions #--force-reinstall
%pip install git+https://github.com/mkrphys/ipython-tikzmagic.git
%load_ext tikzmagic

notebookname = "ps09.ipynb"

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/mkrphys/ipython-tikzmagic.git to /private/var/folders/m8/4s16jlw13wj7flc20jyzpbn40000gn/T/pip-req-build-ht_fr76l
  Running command git clone --filter=blob:none --quiet https://github.com/mkrphys/ipython-tikzmagic.git /private/var/folders/m8/4s16jlw13wj7flc20jyzpbn40000gn/T/pip-req-build-ht_fr76l
  Resolved https://github.com/mkrphys/ipython-tikzmagic.git to commit f9357cc86ea5848947f3d189ee736031bb64890d
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import sys
import control as ct
import numpy as np

sys.path.append(os.path.join(os.getcwd().strip(notebookname), 'utils/'))

from library_tools import library_tf_SISO

np.set_printoptions(suppress=True, precision=3)

### $H^\infty$ norm

In this box we will define the $H^\infty$ norm that we will use to check
the stability of our systems. What the  $H^\infty$ norm essentially does is, 
that it looks for the highest singlular value across all frequencies.
Below you can find the definition of the  $H^\infty$ norm. Feel free to have a 
look.

In [5]:
def hinf_norm(system: ct.TransferFunction) -> int:
    """Calculate the h_inf_norm of a given systems transfer function."""
    # Calculate frequency response over a wide range of frequencies
    omega = np.linspace(-4, 4, 1000)
    H = system(omega * 1j)
    # Calculate all the singular values after checking for MIMO
    if system.ninputs > 1 or system.noutputs > 1:
        singular_values = [np.linalg.svd(H[..., i], compute_uv=False)
                        for i in range(len(omega))]
    else:
        singular_values = [np.absolute(H[... , i]) for i in range(len(omega))]
    # Return the highest singular value
    return np.vstack(singular_values).max()

# Exercise 1: Small gain Theorem

Consider the following System with three plants(P0, P1, P2)
Write a function to check if the system is stable using the small gain theorem.
You have the following functions at hand:
- __np.hinf_norm(transfer_function)__ which calculates the h infinity norm from a given lti system e.g. _np.hinf_norm([1],[1,1]) = __
- __get_transfer_function(system)__ which gives you the transfer function of the subsystems e.g. get_transfer_function(0) for P0.

Write a function that returns a boolean depending on the stability of the system.




In [7]:
def small_gain_theorem(P0: int, P1: int, P2: int) -> bool:
    """Checks if the system is stable depending on the given subsystems."""
    is_stable = False
    # TODO: Write a function to check for the stability of the system.
    
    return is_stable



# Exercise 2: Stability
Think of a cool exercise.

In [ ]:
def stability_check() -> bool:
    is_stable = True
    
    #TODO: Write a function that checks the internal and external stability of the system.

    return is_stable

# Exercise 3: Performance Robustness
Think of a cool exercise.